In [ ]:
from keras.models import load_model
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import numpy as np
import argparse
import pickle
import cv2
import os
import time 
from keras.models import load_model
from collections import deque
import pandas as pd

In [ ]:
# Define the path to your video and your converted TFLite model
V_DIR = '/home/kronbii/dataset/Violence'
NV_DIR = '/home/kronbii/dataset/NonViolence'
V_OUTPUT_DIR = '/home/kronbii/repos/content-violence-detection/output/keras/V'
NV_OUTPUT_DIR = '/home/kronbii/repos/content-violence-detection/output/keras/NV'
MODEL_DIR = '/home/kronbii/repos/content-violence-detection/original_models/modelnew.h5'
CONFIDENCE = 0.400

In [ ]:

def process_dir(input_dir, base_output_dir):
    try:
        try:
            #fig=plt.figure(figsize=(16, 30))
            if not os.path.exists(base_output_dir):
                os.mkdir(base_output_dir)
        except Exception as e:
            print(f"Error creating output directory: {e}")
            return
    
        try:
            model = load_model(MODEL_DIR)
        except Exception as e:
            print(f"Error loading TFLite model: {e}")
            return
        
            # --- 2. Get a list of all video files in the input directory ---
        try:
            video_files = [f for f in os.listdir(input_dir) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
            if not video_files:
                print(f"No video files found in '{input_dir}'.")
                return
            print(f"Found {len(video_files)} videos to process.")
        except FileNotFoundError:
            print(f"Fatal Error: Input directory not found at '{input_dir}'")
            return
    
        for video_filename in video_files:
            csv_file_path = os.path.join(base_output_dir, f"{os.path.splitext(video_filename)[0]}.csv")
            if os.path.exists(csv_file_path):
                print(f"Skipping {video_filename}: CSV already exists at {csv_file_path}")
                continue
            video_path = os.path.join(input_dir, video_filename)
            # Create a unique output directory for this video's results
            video_name_without_ext = os.path.splitext(video_filename)[0]
            output_path_for_video = os.path.join(base_output_dir)
            
            
            print(f"\n--- Processing video: {video_filename} ---")
            print(f"Output will be saved to: {output_path_for_video}")
            
            try:
                if not os.path.exists(output_path_for_video):
                    os.makedirs(output_path_for_video)
            except Exception as e:
                print(f"Error creating output directory for video: {e}")
                return # Skip to the next video
            
            try:
                Q = deque(maxlen=128)
                vs = cv2.VideoCapture(video_path)
    
                if not vs.isOpened():
                    print(f"Error: Could not open video file: {video_path}")
                    return
    
                (W, H) = (None, None)
                count = 0
                preds_list = []
            except Exception as e:
                print(f"Error initializing video processing: {e}")
                return
    
            while True:
                try:
                    # read the next frame from the file
                    (grabbed, frame) = vs.read()
                    if not grabbed:
                        break
                    
                    # if the frame dimensions are empty, grab them
                    if W is None or H is None:
                        (H, W) = frame.shape[:2]
            
                    output = frame.copy()
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame = cv2.resize(frame, (128, 128)).astype("float32")
                    frame = frame.reshape(128, 128, 3) / 255
            
                    # Suppress progress bar by setting verbose=0 in model.predict
                    preds = model.predict(np.expand_dims(frame, axis=0), verbose=0)[0]
                    
                    if 'preds_list' not in locals():
                        preds_list = []
                    preds_list.append(preds)
                    
                    Q.append(preds)
                    results = np.array(Q).mean(axis=0)
            
                    i = (preds < CONFIDENCE)[0]
                    label = 'True' if i else 'False'
                    count += 1
                except Exception as e:
                    print(f"Error processing frame {count}: {e}")
                    continue
            try:
                preds_df = pd.DataFrame({
                    'confidence': [float(p[0]) for p in preds_list]
                })
                csv_path = os.path.join(output_path_for_video, f'{video_name_without_ext}.csv')
                preds_df.to_csv(csv_path, index=False)
                print(f"✅ Predictions for {video_filename} saved to {csv_path}")
            except Exception as e:
                print(f"Error saving predictions to CSV for {video_name_without_ext}: {e}")
            finally:
                vs.release()
    except Exception as e:
        print(f"ERROR AKHO MANYUKE: {e}")       
    print("\n--- All videos processed. ---")
                

In [ ]:
process_dir(V_DIR, V_OUTPUT_DIR)
process_dir(NV_DIR, NV_OUTPUT_DIR)